## Capítulo 10 - Agregação de dados e operações em grupos

Classificar um conjunto de dados e aplicar uma função a cada grupo, seja uma agregação ou uma transformação, com frequência é um componente essencial em um fluxo de trabalho de análise de dados.

Após carregar, mesclar e preparar um conjunto de dados, talvez seja necessário calcular estatísticas de grupos ou, provavelmente, tabelas pivôs visando a relatórios e visualizações. O pandas oferece uma interface flexível 'groupby', que permite manipular e resumir conjuntos de dados de forma natural.

Neste capítulo será mostrado: 
    
    -> Separar um objeto do pandas em partes usando uma ou mais chaves (na forma de funções, arrays ou nomes de colunas de um DataFrame);
    
    -> Calcular estatísticas de resumo para grupos, como contador, média ou desvio-padrão, ou aplicar uma função definida pelo usuário;
    
    -> Aplicar transformações em grupos ou fazer outras manipulações como normalização, regressão linear, classificação ou seleção de subconjuntos;
    
    -> Calcular tabelas pivôs e tabulações cruzadas;
    
    -> Fazer análise de quantis e outras análises estatísticas de grupos;

### 10.1 - Funcionamento de GroupBy

Na primeira etapa do processo, os dados contidos em um objeto do pandas, seja uma Series, um DataFrame ou algo diferente, são separados (split) em grupos, com base em uma ou mais chaves especificadas. A separação é feita em uma eixo em particular de um objeto.

Por exemplo, um DataFrame pode ser agrupado com base em suas linhas (axis=0) ou suas colunas (axis=1). Depois, uma função é aplicada (apply) em cada grupo, gerando um novo valor. Por fim, os resultados de todas essas aplicações de função são combinados (combine) formando um objeto resultante. O formato desse objeto em geral dependerá do que está sendo feito com os dados.

###### Cada chave de grupo pode assumir diversas formas, e as chaves não precisam ser todas do mesmo tipo:
    
    -> Um lista ou um array de valores de mesmo tamanho que o eixo sendo agrupado;
    
    -> Um valor indicando um nome de coluna em um DataFrame;
    
    -> Um dicionário ou um Series especificando uma correspondência entre os valores do eixo sendo agrupado e 
    os nomes dos grupos;
    
    -> Uma função a ser chamada no índice do eixo ou os rótulos individuais no índice.

In [1]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_columns = 20
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

In [2]:
import numpy as np
import pandas as pd

###### Um pequeno conjunto de dados tabular na forma de um DataFrame:

In [3]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'], 
                   'key2' : ['one', 'two', 'one', 'two', 'one'], 
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023
4,a,one,1.965781,1.246435


###### Suponha que quiséssemos calcular a média da coluna 'data1' usando os rótulos de 'key1'.

Há várias maneiras de fazer. Uma delas é acessar 'data1' e chamar 'groupby' com a coluna (uma Series) em 'key1':

In [4]:
grouped = df['data1'].groupby(df['key1'])
grouped

###### A variável 'grouped' agora é um objeto 'GroupBy'

Por exemplo, calcular as médias dos grupos, pode-se chamar o método 'man' de 'GroupBy'

In [5]:
grouped.mean()

key1
a    0.746672
b   -0.537585
Name: data1, dtype: float64

Se, em vez disso, tivéssemos passado vários arrays na forma de uma lista, obteríamos um resultado resultante:

In [6]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.880536
      two     0.478943
b     one    -0.519439
      two    -0.555730
Name: data1, dtype: float64

Nesse último caso, agrupamos os dados usando duas chaves, e a Series resultante agora tem um índice hierárquico constituído dos pares de chave únicos observados:

In [7]:
means.unstack()

key2,one,two
key1,,
a,0.880536,0.478943
b,-0.519439,-0.555730


No exemplo, a seguir, todas as chaves de grupo são Series, embora pudessem ser qualquer array do tamanho correto:

In [8]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])

years = np.array([2005, 2005, 2006, 2005, 2006])

In [9]:
df['data1'].groupby([states, years]).mean()

California  2005    0.478943
            2006   -0.519439
Ohio        2005   -0.380219
            2006    1.965781
Name: data1, dtype: float64

Com frequência, as informações de agrupamento se encontram no mesmo DataFrame em que estão os dados com os quais, quer trabalhar. Nesse caso, é possível passar os nomes das colunas (sejam elas strings, números ou outros objetos Python) como as chaves de grupo:

In [10]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.746672,0.910916
b,-0.537585,0.525384


In [11]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.880536  1.319920
     two   0.478943  0.092908
b    one  -0.519439  0.281746
     two  -0.555730  0.769023

###### Um método em geral últil de GroupBy é 'size', que devolve uma Series contendo os tamanhos dos grupos:

In [12]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

###### Perceba no exemplo acima que qualquer valor ausente em uma chave de grupo será excluído do resultado.

### Iterando por grupos

O objeto 'GroupBy' aceita iteração, gerando uma sequência de tuplas de 2 contendo o nome do grupo, junto com a porção de dados. Considere o seguinte: 

In [13]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.204708  1.393406
1    a  two  0.478943  0.092908
4    a  one  1.965781  1.246435
b
  key1 key2     data1     data2
2    b  one -0.519439  0.281746
3    b  two -0.555730  0.769023


###### No caso de várias chaves, o primeiro elemento da tupla será uma tupla de valores de chaves:

In [14]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.204708  1.393406
4    a  one  1.965781  1.246435
('a', 'two')
  key1 key2     data1     data2
1    a  two  0.478943  0.092908
('b', 'one')
  key1 key2     data1     data2
2    b  one -0.519439  0.281746
('b', 'two')
  key1 key2    data1     data2
3    b  two -0.55573  0.769023


É clro que pode optar por fazer o que quiser com as porções de dados. Uma receita que talvez seja últil é gerar um dicionário de porções de dados usando uma só linha de código:

In [15]:
pieces = dict(list(df.groupby('key1')))

In [16]:
pieces['b']

,key1,key2,data1,data2
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023


Por padrão, 'groupby' agrupa em 'axis=0', mas podemos agrupar em qualquer um dos outros eixos. Por exemplo, é possível agrupar as colunas de nosso exemplo com 'df' de acordo com o dtype:

In [17]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [18]:
grouped = df.groupby(df.dtypes, axis=1)

In [19]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.204708  1.393406
1  0.478943  0.092908
2 -0.519439  0.281746
3 -0.555730  0.769023
4  1.965781  1.246435
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### Selecionando uma coluna ou um subconjunto de colunas

Indexar um objeto 'GroupBy' criado a partir de um DataFrame com um nome de coluna ou um array de nomes de coluna tem o efeito de criar subconjuntos de colunas para agregação.

In [20]:
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]

Açucar sintático para:

In [21]:
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key2'])

Particularmente para conjuntos grandes de dados, fazer agregações somente de algumas colunas pode ser desejável. Por exemplo, no conjunto de dados anterior, para calcular as médias apenas da coluna data2 e obter o resultado na forma de um DataFrame, poderíamos escrever o seguinte:

In [22]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   1.319920
     two   0.092908
b    one   0.281746
     two   0.769023

O objeto devolvido por essa operação de indexação é um DataFrame agrupado se uma lista ou um array for passado ou uma Series agrupadas se um único nome de coluna for passado como um escalar:

In [23]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [24]:
s_grouped.mean()

key1  key2
a     one     1.319920
      two     0.092908
b     one     0.281746
      two     0.769023
Name: data2, dtype: float64

### Agrupando com dicionários e Series

Informações de agrupamento podem existir em uma forma que não seja um array.

Vamos considerar outro DataFrame como exemplo:

In [25]:
people = pd.DataFrame(np.random.randn(5, 5), 
                     columns=['a', 'b', 'c', 'd', 'e'], 
                     index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

In [26]:
people.iloc[2:3, [1, 2]] = np.nan # Acrescenta alguns valores NA
people

,a,b,c,d,e
Joe,1.007189,-1.296221,0.274992,0.228913,1.352917
Steve,0.886429,-2.001637,-0.371843,1.669025,-0.438570
Wes,-0.539741,NaN,NaN,-1.021228,-0.577087
Jim,0.124121,0.302614,0.523772,0.000940,1.343810
Travis,-0.713544,-0.831154,-2.370232,-1.860761,-0.860757


Suponha agora que haja uma correspondência de grupos para as colunas e queremos somá-las por grupo: 

In [27]:
mapping={'a':'red', 'b':'red', 'c':'blue', 
         'd':'blue', 'e':'red', 'f':'orange'}
mapping

{'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}

Poderíamos construir um array a partir desse dicionário e passá-lo para groupby mas, em vez disso, podemos simplesmente passar o dicionário (inclui a chave 'f' para enfatizar que chaves de agrupamento não usadas não são são um problema):

In [28]:
by_column = people.groupby(mapping, axis=1)

In [29]:
by_column.sum()

,blue,red
Joe,0.503905,1.063885
Steve,1.297183,-1.553778
Wes,-1.021228,-1.116829
Jim,0.524712,1.770545
Travis,-4.230992,-2.405455


A mesma funcionalidade vale para Series, que pode ser vista como um mapeamento de tamanho fixo:

In [30]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [31]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### Agrupando com funções

Usar funções Python é uma forma mais genérica de definir um mapeamento de grupos, em comparação com um dicionário ou uma Series. Qualquer função passada como uma chave de grupo será chamada em uma vez por valor de índice, com os valores de retorno usados como os nomes dos grupos. De modo mais concreto, considere o DataFrame de exemplo da seção anterior, que tem os primeiros nomes das pessoas como valores de índice. Suponha que quiséssemos agrupar pelo tamanho dos nomes; embora podéssemos calcular um array de tamanhos de strings, será mais fácil simplesmente passar a função 'len': 

In [32]:
people.groupby(len).sum()

,a,b,c,d,e
3,0.591569,-0.993608,0.798764,-0.791374,2.119639
5,0.886429,-2.001637,-0.371843,1.669025,-0.438570
6,-0.713544,-0.831154,-2.370232,-1.860761,-0.860757


Misturar funções com arrays, dicionários ou Series não é um problema, pois tudo será convertido para arrays internamente:

In [33]:
key_list = ['one', 'one', 'one', 'two', 'two']
key_list

['one', 'one', 'one', 'two', 'two']

In [34]:
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.539741 -1.296221  0.274992 -1.021228 -0.577087
  two  0.124121  0.302614  0.523772  0.000940  1.343810
5 one  0.886429 -2.001637 -0.371843  1.669025 -0.438570
6 two -0.713544 -0.831154 -2.370232 -1.860761 -0.860757

### Agrupando por níveis de índice

Um último recurso conveniente para conjuntos de dados hierarquicamente indexados é a capacidade de agregar usando um dos níveis de índice de um eixo. Vamos observar um exemplo: 

In [35]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'], 
                                     [1, 3, 5, 1, 3]], 
                                    names=['city', 'tenor'])
columns

MultiIndex([('US', 1),
            ('US', 3),
            ('US', 5),
            ('JP', 1),
            ('JP', 3)],
           names=['city', 'tenor'])

In [36]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

city         US                            JP          
tenor         1         3         5         1         3
0      0.560145 -1.265934  0.119827 -1.063512  0.332883
1     -2.359419 -0.199543 -1.541996 -0.970736 -1.307030
2      0.286350  0.377984 -0.753887  0.331286  1.349742
3      0.069877  0.246674 -0.011862  1.004812  1.327195

Para agrupar por nível, passe o número ou o nome do nível usando o agrupamento nomeado level:

In [37]:
hier_df.groupby(level='city', axis=1).count()

city,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2 Agregação de dados

As agregações referem-se a qualquer transformação de dados que gere valores escalares a partir de arrays.Os exemplos anteriores usaram várias delas, incluindo 'mean', 'count', 'min' e 'sum'. Talvez esteja se perguntando o que acontece quando chamamos 'mean()' em um objeto GroupBy. Muitas agregações comuns, como aquelas que se encontram na tabela 10.1, têm implementações otimizadas. Contudo não estamos limitados a apenas esse conjunto de métodos.

###### Tabela 10.1 - Métodos otimizados de groupby

count => Número de valores diferentes de NA no grupo.

sum => Soma dos valores diferentes de NA.

mean => Média dos valores diferentes de NA.

median => Mediana aritémica dos valores diferentes de NA.

std, var => Desvio-padrão não tendencioso (determinador n - 1) e variância.

min, max => Mínimo e máximo entre os valores diferentes de NA.

prod => Produto dos valores diferentes de NA.

first, last => Primeiro e último dos valores diferentes de NA.



Pode usar próprias agregações, e além disso, chamar qualquer método que também esteja definido no objeto agrupado. Por exemplo, talvez se lembre que quantile calcula os quantis de amostragem em uma Series ou em colunas de um DataFrame. 

Embora 'quantile' não seja explicitamente implementado para 'GroupBy', é um método de Series, e, desse modo, está disponível para ser usado. Internamente, GroupBy fatia a Series de modo eficiente, chama 'piece.quantile(0.9)' para cada parte e então reúne os resultados no objeto resultante:

In [38]:
df

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023
4,a,one,1.965781,1.246435


In [39]:
grouped = df.groupby('key1')
grouped

In [40]:
grouped['data1'].quantile(0.9)

key1
a    1.668413
b   -0.523068
Name: data1, dtype: float64

Para usar as próprias funções de agregações passe qualquer função qie agregue um array para o método 'agregate' ou 'agg'.

In [41]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [42]:
grouped.agg(peak_to_peak)

/tmp/ipykernel_6549/4193407341.py:1: FutureWarning: ['key2'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  grouped.agg(peak_to_peak)


,data1,data2
key1,,
a,2.170488,1.300498
b,0.036292,0.487276


Pode notar que alguns métodos como 'describe' também funcionam, embora não sejam de agregação, estritamente falando:

In [43]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.746672  1.109736 -0.204708  0.137118  0.478943  1.222362   
b      2.0 -0.537585  0.025662 -0.555730 -0.546657 -0.537585 -0.528512   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.965781   3.0  0.910916  0.712217  0.092908  0.669671  1.246435   
b    -0.519439   2.0  0.525384  0.344556  0.281746  0.403565  0.525384   

                          
           75%       max  
key1                      
a     1.319920  1.393406  
b     0.647203  0.769023

### Aplicação de função nas colunas e aplicação de várias funções

Vamos voltar ao conjunto de dados de gorjetas de exemplos americanos. Depois de carregá-lo com 'read_csv', adicionamos uma coluna "tip_pct" de porcentagem de gorjetas:

In [44]:
tips = pd.read_csv('examples/tips.csv')

In [45]:
# Acrescenta a porcentagem de gorjeta sobre o total da conta
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [46]:
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


Conforme já vimos, fazer a agregação em uma Series ou em todas as colunas de um DataFrame é uma questão de usar o 'agregate' com a função desejada ou chamar um método como 'mean' ou 'std'. Entretanto, talvez queira fazer a agregação usando uma função diferente, conforme a coluna, ou utilizando várias funções de uma só vez. Felizmente isso é possível, onde será demonstrado por meio de uma série de exemplos. Inicialmente será agrupado tips de acordo com 'day' e 'smoker':

In [47]:
grouped = tips.groupby(['day', 'smoker'])

Observe que, para estatísticas descritivas como aquelas da 'Tabela 10.1', podemos passar o nome da função como uma string:

In [48]:
grouped_pct = grouped['tip_pct']

In [49]:
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

Se uma lista de funções ou de nomes de função for especificada, terá de volta um DataFrame com os nomes das colunas obtidos com base nas funções:

In [50]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

Nesse exemplo, passamos uma lista de funções de agregação para agg a fim de serem avaliadas de modo independente nos grupos de dados.

Não precisa aceitar os nomes que GroupBy dá as colunas; merece destaque o fato de as funções 'lambda' terem o nome '<lambda', o que as deixa difíceis de serem identificadas. Desse modo, se passar uma lista de tuplas (name, function), o primeiro elemento de cada tupla será usado como os nomes das colunas do DataFrame(uma lista de 2 elementos como um mapeamento ordenado):

In [51]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

Com um DataFrame, temos mais opções, pois podemos especificar uma lista de funções a serem aplicadas em todas as colunas, ou diferentes funções por coluna.

Para começar, suponha que quisêssemos calcular as mesmas três estatísticas para as colunas 'tip_pct' e 'total_bill':

In [52]:
functions = ['count', 'mean', 'max']

In [53]:
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

/tmp/ipykernel_6549/1930447398.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result = grouped['tip_pct', 'total_bill'].agg(functions)


tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

Como podemos ver, o DataFrame resultante tem colunas hierárquicas - as mesmas que obteríamos se agressássemos cada coluna separadamente e usássemos 'concat' para unir os resultados, utilizando os nomes das colunas como um argumento 'keys':

In [54]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

Como antes, uma lista de tuplas com nomes personalizados pode ser passada:

In [55]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
ftuples

[('Durchschnitt', 'mean'),
 ('Abweichung',
  <function numpy.var(a, axis=None, dtype=None, out=None, ddof=0, keepdims=<no value>, *, where=<no value>)>)]

In [56]:
grouped['tip_pct', 'total_bill'].agg(ftuples)

/tmp/ipykernel_6549/514018043.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped['tip_pct', 'total_bill'].agg(ftuples)


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

Suponha agora que quisesse aplicar funções possivelmente distintas em uma ou mais colunas. Para isso, passe um dicionário para agg, contendo um mapeamento de nomes de colunas para qualquer uma das especificações de função listadas até agora:

In [57]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [58]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'], 
             'size': 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

Um DataFrame terá colunas hierárquicas somente se várias funções forem aplicadas em no mínimo uma coluna.

### Devolvendo dados agregados sem índices de linha

Em todos os exemplos até agora, os dados agregados retornavam com um índice, possivelmente hierárquicos, composto de combinações únicas de chaves de grupo.

Com isso nem sempre é desejável, podemos desativar esse comportamento na maioria dos casos, passando 'as_index=False' para groupby:

In [59]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


Claro que sempre é possível obter o resultado nesse formato chamando 'reset_index' nele. Usar o método 'as_index=False' evita alguns processamentos desnecessários.

## 10.3 Método apply:separar-aplicar-combinar genérico
    
O método de propósito mais geral de GroupBy é 'apply', que será o assunto do restante desta seção.'Apply' separa o objeto sendo manipulado em partes, chama a função recebida em cada parte e, em seguida, tenta concatená-las.

Retornando ao conjunto de dados anterior de gorjetas, suponha que quiséssemos selecionar os cinco primeiros valores de 'tip_pct' por grupo. Inicialmente, escreva uma função que selecione as linhas com os maiores valores em uma coluna em particular:

In [60]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [61]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


Se agruparmos de acordo com smoker, por exemplo, e chamarmos 'apply' com essa função, teremos o seguinte:

In [62]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

o que aconteceu nesse caso? A função 'top' é chamada em cada grupo de linhas do DataFrame; então os resultados são unidos com 'pandas.concat', atribuindo os nomes dos grupos como rótulos para cada parte. Assim, o resultado tem um índice hierárquico cujo nível mais interno contém valores de índice do DataFrame original.

Se passar uma função para 'apply' que aceite outros argumentos ou argumentos nomeados, esses poderão ser passados depois da função:

In [63]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [64]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [65]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

Em GroupBy, quando chamamos um método como 'describe', esse será, na verdade, apenas um atalho para:

In [66]:
f = lambda x: x.describe()
grouped.apply(f)

total_bill       tip  size   tip_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]

### Suprimindo as chaves de grupo

Nos exemplos anteriores, vemos que o objeto resultante tem um índice hierárquico composto das chaves de grupo, junto com os índices de cada parte do objeto original. Podemos desativar isso passando 'groupby=False' para 'groupby':

In [67]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### Análise de quantis e de buckets

O pandas tem algumas ferramentas, em particular 'cut' e 'qcut', para fatiar dados em buckets, como compartimentos (bins) de sua preferência, ou por quantis da amostra. Combinar essas funções com 'groupby' faz com que seja conveniente realizar análises de buckets ou de quantis em um conjunto de dados. Considere um conjunto de dados aleatório simples e uma classificação em buckets de mesmo tamanho usando 'cut':

In [68]:
frame = pd.DataFrame({'data1' : np.random.randn(1000), 
                      'data2' : np.random.randn(1000)})
frame

,data1,data2
0,-0.919262,1.165148
1,-1.549106,-0.621249
2,0.022185,-0.799318
3,0.758363,0.777233
4,-0.660524,-0.612905
...,...,...
995,-0.459849,-0.574654
996,0.333392,0.786210
997,-0.254742,-1.393822
998,-0.448301,0.359262


In [69]:
quartiles = pd.cut(frame.data1, 4)
quartiles

0       (-1.23, 0.489]
1      (-2.956, -1.23]
2       (-1.23, 0.489]
3       (0.489, 2.208]
4       (-1.23, 0.489]
            ...       
995     (-1.23, 0.489]
996     (-1.23, 0.489]
997     (-1.23, 0.489]
998     (-1.23, 0.489]
999    (-2.956, -1.23]
Name: data1, Length: 1000, dtype: category
Categories (4, interval[float64, right]): [(-2.956, -1.23] < (-1.23, 0.489] < (0.489, 2.208] < (2.208, 3.928]]

In [70]:
quartiles[:10]

0     (-1.23, 0.489]
1    (-2.956, -1.23]
2     (-1.23, 0.489]
3     (0.489, 2.208]
4     (-1.23, 0.489]
5     (0.489, 2.208]
6     (-1.23, 0.489]
7     (-1.23, 0.489]
8     (0.489, 2.208]
9     (0.489, 2.208]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-2.956, -1.23] < (-1.23, 0.489] < (0.489, 2.208] < (2.208, 3.928]]

O objeto Categorical devolvido por cut pode ser passado diretamente para groupby. Assim, podemos calcular um conjunto de estatísticas para a coluna 'data2' da seguinte maneira:

In [71]:
def get_stats(group):
    return {'min': group.min(), 'max':group.max(),
            'count': group.count(), 'mean': group.mean()}

In [72]:
grouped = frame.data2.groupby(quartiles)

In [73]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-2.956, -1.23]",-3.399312,1.670835,95.0,-0.039521
"(-1.23, 0.489]",-2.989741,3.260383,598.0,-0.002051
"(0.489, 2.208]",-3.745356,2.954439,297.0,0.081822
"(2.208, 3.928]",-1.929776,1.765640,10.0,0.024750


Esses buckets eram de mesmo tamanho; para calcular buckets de mesmo tamanho em quantis de amostra, utilize 'qcut'. Usarei 'Labels=False' para obter somente os números dos quantis:

In [74]:
# Devolve os números dos quantis
grouping = pd.qcut(frame.data1, 10, labels=False)
grouping

0      1
1      0
2      5
3      7
4      2
      ..
995    3
996    6
997    4
998    3
999    0
Name: data1, Length: 1000, dtype: int64

In [75]:
grouped = frame.data2.groupby(grouping)

In [76]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-3.399312,1.670835,100.0,-0.049902
1,-1.950098,2.628441,100.0,0.030989
2,-2.925113,2.527939,100.0,-0.067179
3,-2.315555,3.260383,100.0,0.065713
4,-2.047939,2.074345,100.0,-0.111653
5,-2.989741,2.184810,100.0,0.052130
6,-2.223506,2.458842,100.0,-0.021489
7,-3.056990,2.954439,100.0,-0.026459
8,-3.745356,2.735527,100.0,0.103406


### Exemplo: preenchendo valores ausentes com valores específicos de grupo
    
Ao limpar dados ausentes, em alguns casos, você substituirá os dados observados usando 'dropna', porém, em outros, talvez queira representar valores nulos (NA), isto é, preenchê-los, com um valor fixo ou outro valor derivado dos dados, 'fillna' é a ferramenta correta a ser usada; por exemplo, preencherei a seguir 'NA' com a média:

In [77]:
s = pd.Series(np.random.randn(6))
s

0    0.678661
1   -0.125921
2    0.150581
3   -0.884475
4   -0.620521
5    0.227290
dtype: float64

In [78]:
s[::2] = np.nan
s

0         NaN
1   -0.125921
2         NaN
3   -0.884475
4         NaN
5    0.227290
dtype: float64

In [79]:
s.fillna(s.mean())

0   -0.261035
1   -0.125921
2   -0.261035
3   -0.884475
4   -0.261035
5    0.227290
dtype: float64

Suponha que seja necessário que o valor de preenchimento varie conforme o grupo. Um modo de fazer isso é agrupar os dados e usar 'apply' com uma função que chame 'fillna' em cada porção de dados. Eis alguns dados de exemplo dos estados 'norte-americanos', divididos em regiões 'leste' e 'oeste':

In [80]:
states = ['Ohio', 'New York', 'Vermont', 'Florida', 
          'Oregon', 'Nevada', 'California', 'Idaho']
states

['Ohio',
 'New York',
 'Vermont',
 'Florida',
 'Oregon',
 'Nevada',
 'California',
 'Idaho']

In [81]:
group_key = ['East'] * 4 + ['West'] * 4
group_key

['East', 'East', 'East', 'East', 'West', 'West', 'West', 'West']

In [82]:
data = pd.Series(np.random.randn(8), index=states)
data

Ohio          0.922264
New York     -2.153545
Vermont      -0.365757
Florida      -0.375842
Oregon        0.329939
Nevada        0.981994
California    1.105913
Idaho        -1.613716
dtype: float64

Observe que a sintaxe ['East'] * 4 gera uma lista contendo quatro cópias dos elementos em ['East']. Somar listas faz com que elas sejam concatenadas.

In [83]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio          0.922264
New York     -2.153545
Vermont            NaN
Florida      -0.375842
Oregon        0.329939
Nevada             NaN
California    1.105913
Idaho              NaN
dtype: float64

In [84]:
data.groupby(group_key).mean()

East   -0.535707
West    0.717926
dtype: float64

Podemos preencher os valores NA usando as médias dos grupos, assim:

In [85]:
fill_mean = lambda g: g.fillna(g.mean())

In [86]:
data.groupby(group_key).apply(fill_mean)

Ohio          0.922264
New York     -2.153545
Vermont      -0.535707
Florida      -0.375842
Oregon        0.329939
Nevada        0.717926
California    1.105913
Idaho         0.717926
dtype: float64

Em outro caso, você pode ter valores de preenchimento predefinidos em seu código, que variem conforme o grupo. Como os grupos têm um atributo 'name' definido internamente, podemos usá-lo:

In [87]:
fill_values = {'East': 0.5, 'West': -1}

In [88]:
fill_func = lambda g: g.fillna(fill_values[g.name])

In [89]:
data.groupby(group_key).apply(fill_func)

Ohio          0.922264
New York     -2.153545
Vermont       0.500000
Florida      -0.375842
Oregon        0.329939
Nevada       -1.000000
California    1.105913
Idaho        -1.000000
dtype: float64

### Exemplo: amostragem aleatória e permutação
    
Suponha que quiséssemos sortear uma amostra aleatória (com ou sem substituição) a partir de um conjunto de dados grande, visando a uma simulação de Monte Carlo ou outra aplicação. Há algumas maneiras de fazer os 'Sorteios'; usaremos a seguir o método 'sample' de Series:

Para uma demonstração, eis uma forma de construir um baralho em inglês:

In [90]:
# Copas (Heart), Espada(Spades), Paus(Clubs) e Ouro(Diamonds)
suits=['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)
deck

AH      1
2H      2
3H      3
4H      4
5H      5
       ..
9D      9
10D    10
JD     10
KD     10
QD     10
Length: 52, dtype: int64

Agora temos uma Series de tamanho 52 cujo índice contém os nomes das cartas e os valores são aqueles usados no BlackJack e em outros jogos.

In [91]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

Com base no que dissemos antes, sortear uma mão de cinco cartas do baralho poderia ser escrito da seguinte maneira:

In [92]:
def draw(deck, n=5):
    return deck.sample(n)

In [93]:
draw(deck)

AD     1
8C     8
5H     5
KC    10
2C     2
dtype: int64

Suponha que quiséssemos duas cartas aleatórias de cada naipe. Como o naipe é o último caractere do nome de cada carta, podemos fazer agrupamentos com base nisso e usar 'apply':

In [94]:
get_suit = lambda card: card[-1] # A última letra é o naipe

In [95]:
deck.groupby(get_suit).apply(draw, n=2)

C  2C     2
   3C     3
D  KD    10
   8D     8
H  KH    10
   3H     3
S  2S     2
   4S     4
dtype: int64

De modo alternativo, poderíamos escrever:

In [96]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

KC    10
JC    10
AD     1
5D     5
5H     5
6H     6
7S     7
KS    10
dtype: int64

### Exemplo: média ponderada de grupos e correlação
    
No paradigma separar-aplicar-combinar de groupby, operações entre colunas em um DataFrame ou em duas Series, como uma média podenrada de grupos, são possíveis. Como exemplo, considere o conjunto de dados a seguir contendo chaves de grupo, valores e alguns pesos:

In [97]:
df = pd.DataFrame({'category':['a', 'a', 'a', 'a', 
                               'b', 'b', 'b', 'b'], 
                   'data': np.random.randn(8), 
                   'weights': np.random.rand(8)})
df

,category,data,weights
0,a,1.561587,0.957515
1,a,1.219984,0.347267
2,a,-0.482239,0.581362
3,a,0.315667,0.217091
4,b,-0.047852,0.894406
5,b,-0.454145,0.918564
6,b,-0.556774,0.277825
7,b,0.253321,0.955905


A média ponderada dos grupos por category seria então:

In [98]:
grouped = df.groupby('category')

In [99]:
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])

In [100]:
grouped.apply(get_wavg)

category
a    0.811643
b   -0.122262
dtype: float64

Como outro exemplo, considere um conjunto de dados financeiros, originalmente obtido do Yahoo! Finance, contendo os preços de algumas ações no final do dia e o índice S&P 500 (o símbolo SPX):

In [101]:
close_px = pd.read_csv('examples/stock_px.csv', parse_dates=True, index_col=0)
close_px.head()

,AAPL,MSFT,XOM,SPX
2003-01-02,7.40,21.11,29.22,909.03
2003-01-03,7.45,21.14,29.24,908.59
2003-01-06,7.45,21.52,29.96,929.01
2003-01-07,7.43,21.93,28.95,922.93
2003-01-08,7.28,21.31,28.83,909.93


In [102]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [103]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


Uma tarefa interessante seria calcular um DataFrame constituído das correlações anuais entre retornos diários (calculados a partir da mudança percentuais) e SPX.

Como forma de fazer isso, inicialmente criaremos uma função que calcula a correlação aos pares entre cada coluna e a coluna 'SPX':

In [104]:
spx_corr = lambda x: x.corrwith(x['SPX'])

Em seguida, calculamos as mudanças percentuais em 'close_px' usando 'pct_change'

In [105]:
rets = close_px.pct_change().dropna()

Por fim, agrupamos essas mudanças percentuais por ano; esse valor pode ser extraído de cada rótulo de linha com uma função de uma linha que devolve o atributo 'year' de cada rótulo 'datetime':

In [106]:
get_year = lambda x:x.year

In [107]:
by_year = rets.groupby(get_year)

In [108]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


Também poderíamos calcular as correlações entre as colunas. A seguir, calcularemos a correlação anual entre a Apple e a Microsoft:

In [109]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### Exemplo: regressão linear nos grupos
    
Seguindo a mesma temática do exemplo anterior, podemos utilizar 'groupby' para realizar análises estatísticas mais complexas nos grupos, desde que a função devolva um objeto do pandas ou um valor escalar. Por exemplo, posso definir a função 'regress' a seguir (usando a biblioteca de econometria 'statsmodels'), que executa uma regressão OLS (Mínimos Quadrados Ordinários) em cada porção de dados:

In [110]:
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

Para executar uma regressão linear anual de 'AAPL' nos retornos de SPX, execute:

In [111]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


### 10.4 Tabelas pivôs e tabulação cruzada

Uma tabela pivô é uma ferramenta de sintetização de dados frequentemente entrada em programas de planilhas e em outros softwares de análise de dados. Ela agrega uma tabela de dados de acordo com uma ou mais chaves, organizando os dados em um retângulo com algumas das chaves de grupo nas linhas e outras nas colunas. As tabelas pivôs em Python com pandas são possíveis por meio do recurso 'groupby' descrito neste capítulo, em conjunto com operações de reformatação que utilizam indexação hierárquica. O DataFrame tem um método 'pivot_table', e há também uma função 'pandas.pivot_table' de nível superior. Além de oferecer uma interface conveniente para 'groupby.pivot_table' pode somar totais parciais, também conhecidos como 'margens(margins)'.

Voltando ao conjunto de dados de gorjetas, suponha que quiséssemos calcular uma tabela de médias de grupos (o tipo de agregação default de 'pivot_table'), organizado por 'day' e 'smoker' nas linhas:

In [112]:
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

Esses dados poderiam ter sido gerados diretamente com 'groupby'. Suponha agora que quiséssemos agregar apenas 'tip_pct' e 'size', e, além disso, agrupar de acordo com 'time'. Colocarei 'smoker' nas colunas da tabela e 'day' nas linhas:

In [113]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], 
                 columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

Poderíamos expandir essa tabela de modo que inclua totais parciais, passando 'margins=True'. Isso tem o efeito de adicionar rótulos 'All' para linhas e colunas, com os valores correspondemos sendo as estatísticas de grupo para todos os dados em uma única camada:

In [114]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], 
                 columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

Nesse exemplo, os valores de 'All' são as médias, sem levar em consideração os fumantes 'versus' os não fumantes (as colunas de All) nem qualquer um dos dois níveis de agrupamento nas linhas (a linha All).

Para utilizar uma função de agregação diferente, passe-a para 'aggfunc'. Por exemplo, 'count' ou 'len' oferecerão uma tabulação cruzada (contador ou frequência) dos tamanhos dos grupos:

In [115]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day', aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

Se algumas combinações forem vazias (ou se forem NA), você poderá passar um 'fill_value':

In [116]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'], 
                 columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
...                      ...       ...       ...       ...
Lunch  3    Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

[21 rows x 4 columns]

Veja a Tabela 10.2 que contém um resumo dos métodos de 'pivot_table'

###### Tabela 10.2 - Opções de pivot_table

values => Nome ou nomes das colunas a serem agregadas, por padrão, agrega todas as colunas numéricas.

index => Nomes das colunas ou outras chaves de grupo para agrupar nas linhas da tabela pivô resultante.

columns => Nomes das colunas ou outras chaves de grupo para agrupar nas colunas da tabela pivô resultante.

aggfunc => Função de agregação ou lista de funções (default é "mean"); pode ser qualquer função válida no contexto de um groupby.

fill_value => Substitui valores ausentes na tabela resultante.

dropna = Se for True, não inclui as colunas cujas entradas sejam todas NA.

margins = Adiciona subtotais para linhas/colunas e um total geral (o default é False).

## 10.4 Conclusão

Dominar as ferramentas de agrupamento de dados do pandas pode ajudar tanto no trabalho de limpeza dos dados como também na modelagem ou na análise estatística